In [3]:
# -------------------------------------------------------
# 📦 1. Import Libraries
# -------------------------------------------------------
import os
%pip install google-cloud-bigquery db-dtypes

from google.cloud import bigquery
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Make sure you've run: gcloud auth application-default login
client = bigquery.Client()

In [5]:
query = """
SELECT *
FROM `scalable-streaming-analytics.streaming_data.processed_events`
WHERE timestamp >= UNIX_SECONDS(CURRENT_TIMESTAMP()) - 3600
"""

df = client.query(query).to_dataframe()
df.head()

I0000 00:00:1743030405.994849  122487 check_gcp_environment.cc:61] BIOS data file does not exist or cannot be opened.


,user_id,event,content_id,timestamp


In [6]:
print("Missing values:\n", df.isnull().sum())
print("\nData types:\n", df.dtypes)

# Convert UNIX timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Encode categorical column
df['event'] = df['event'].astype('category')


Missing values:
 user_id       0.0
event         0.0
content_id    0.0
timestamp     0.0
dtype: float64

Data types:
 user_id       object
event         object
content_id    object
timestamp     object
dtype: object


In [7]:
df.drop_duplicates(inplace=True)


,user_id,event,content_id,timestamp


In [8]:
def remove_timestamp_outliers(df):
    Q1 = df['timestamp'].quantile(0.25)
    Q3 = df['timestamp'].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return df[(df['timestamp'] >= lower_bound) & (df['timestamp'] <= upper_bound)]

df = remove_timestamp_outliers(df)
df.head()


,user_id,event,content_id,timestamp


In [9]:
scaler = MinMaxScaler()
df['timestamp'] = scaler.fit_transform(df[['timestamp']])
df.head()

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
df.to_csv('data/cleaned_events.csv', index=False)